# Meshing a protein using PyGAMer

This tutorial will walk you through the process of using PyGAMer to generate a tetrahedral mesh of the volume surrounding a protein. The protein of interest, PDBID 2JHO, is that of sperm whale myoglobin at 1.4Å resolution.

In [1]:
# Load in PyGAMer and numpy
import pygamer
import numpy as np
# We will use threevis to visualize the mesh in the notebook
import threevis

In [2]:
# Mesh the protein of interest
mesh = pygamer.readPDB_molsurf('../data/2jho.pdb')

When using PyGAMer it's important to initialize the orientation of the simplices. Notably `compute_orientation()` will try to apply a self consistent set of orientations. In many cases, you may wish to ensure that the mesh normals are outward facing. This can be achieved by calling `correctNormals()` which essentially computes the volume of the bounded surface and flips the normals if the volume is negative.

In [3]:
# Compute the normal orientation
mesh.compute_orientation()
mesh.correctNormals()

## Conditioning the protein mesh

In [4]:
# Set selection of all vertices to True so smooth will operate on them.
for v in mesh.vertexIDs:
    v.data().selected = True
# Apply 5 iterations of smoothing
mesh.smooth(max_iter=5, preserve_ridges=True, verbose=True)

Initial Quality: Min Angle = 18.2858, Max Angle = 139.106, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 1:
Min Angle = 16.9737, Max Angle = 131.859, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 2:
Min Angle = 17.7987, Max Angle = 130.704, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 3:
Min Angle = 18.8562, Max Angle = 129.027, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 4:
Min Angle = 19.0814, Max Angle = 127.025, # smaller-than-15 = 0, # larger-than-165 = 0
Iteration 5:
Min Angle = 19.3171, Max Angle = 124.929, # smaller-than-15 = 0, # larger-than-165 = 0


In [5]:
print(F"The mesh currently has {mesh.nVertices} vertices, \
{mesh.nEdges} edges, and {mesh.nFaces} faces.")

The mesh currently has 43080 vertices, 129234 edges, and 86156 faces.


That's a lot of vertices! Here's what the protein mesh looks like at this point. `threevis` provides a convenient way to visualize meshes inside of a jupyter-notebook. It uses arrays of vertex positions, edge and face vertex indices. These arrays can be generated using the `to_ndarray()` function.

In [6]:
protverts, protedges, protfaces = mesh.to_ndarray()
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(protverts, protfaces)
ctx.draw_edges(protverts, protedges)
ctx.display()

Renderer(background='#dddddd', camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(…

### Iterative Coarsening

Depending on your problem, you may wish to coarsen or decimate the mesh. Basically reduce the number of vertices. GAMer offers several functions to help you with this goal.

In [7]:
for i in range(25):
    # Coarsen dense regions of the mesh
    mesh.coarse_dense(rate = 3, numiter = 1)
    # Coarsen flat regions of the mesh
    mesh.coarse_flat(rate = 0.1, numiter = 1)
    mesh.smooth(max_iter = 3, preserve_ridges = True, verbose = False)
    print(F"Iteration {i}: {mesh.nVertices} vertices, \
{mesh.nEdges} edges, and {mesh.nFaces} faces.")

Iteration 0: 29137 vertices, 87405 edges, and 58270 faces.
Iteration 1: 22393 vertices, 67173 edges, and 44782 faces.
Iteration 2: 17819 vertices, 53451 edges, and 35634 faces.
Iteration 3: 14448 vertices, 43338 edges, and 28892 faces.
Iteration 4: 11923 vertices, 35763 edges, and 23842 faces.
Iteration 5: 9933 vertices, 29793 edges, and 19862 faces.
Iteration 6: 8283 vertices, 24843 edges, and 16562 faces.
Iteration 7: 6954 vertices, 20856 edges, and 13904 faces.
Iteration 8: 5857 vertices, 17565 edges, and 11710 faces.
Iteration 9: 4986 vertices, 14952 edges, and 9968 faces.
Iteration 10: 4216 vertices, 12642 edges, and 8428 faces.
Iteration 11: 3572 vertices, 10710 edges, and 7140 faces.
Iteration 12: 3041 vertices, 9117 edges, and 6078 faces.
Iteration 13: 2608 vertices, 7818 edges, and 5212 faces.
Iteration 14: 2214 vertices, 6636 edges, and 4424 faces.
Iteration 15: 1937 vertices, 5805 edges, and 3870 faces.
Iteration 16: 1679 vertices, 5031 edges, and 3354 faces.


RuntimeError: ERROR(getNormal): Orientation undefined, cannot compute normal. Did you call compute_orientation()?

In [ ]:
# Smooth the mesh again.
mesh.smooth(max_iter = 10, preserve_ridges = True, verbose = True)

In [9]:
pygamer.writeOFF('test.off', mesh)

In [ ]:
# Set boundary markers of the mesh to 23
for faceID in mesh.faceIDs:
    faceID.data().marker = 23

In [ ]:
# Get the root metadata
gInfo = mesh.getRoot()
gInfo.ishole = True    # Don't mesh the inside of 
gInfo.marker = -1

In [ ]:
# Center mesh at 0,0,0
center, radius = mesh.getCenterRadius()
mesh.translate(-center)

## Visualizing the mesh with threevis

In [8]:
# Convert mesh to numpy arrays for visualization using threevis
import threevis

protverts, protedges, protfaces = mesh.to_ndarray()
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(protverts, protfaces)
ctx.draw_edges(protverts, protedges)
# Finally display it
ctx.display()

Renderer(background='#dddddd', camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(…

## Now let's construct a bounding box to represent the cytosol around the protein

In [ ]:
# Generate a surrounding box
box = pygamer.surfacemesh.cube(5)

In [ ]:
# Set box boundary markers to 50
for faceID in box.faceIDs:
    faceID.data().marker = 50

In [ ]:
# Get and set box metadata
gInfo = box.getRoot()
gInfo.ishole = False
gInfo.marker = 5

In [ ]:
# Scale the box by 2 times the radius of the protein mesh
box.scale(radius*2)

In [ ]:
boxverts, boxedges, boxfaces = box.to_ndarray()
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(boxverts, boxfaces)
ctx.draw_edges(boxverts, boxedges)
# Finally display it
ctx.display()

In [ ]:
# Draw the protein and surrounding box together
ctx = threevis.Context(width=640, height=480)
ctx.draw_faces(protverts, protfaces)
ctx.draw_edges(boxverts, boxedges)
ctx.display()

Write the meshes to file...
`pygamer.writeOFF('2jho.off', mesh)`
`pygamer.writeOFF('box.off', box)`

## Tetrahedralizing

In [ ]:
# Construct a list of meshes to pass into TetGen
meshes = [mesh, box]

In [ ]:
# Call tetgen to tetrahedralize. The string represents command line arguments passed directly to TetGen.
tetmesh = pygamer.makeTetMesh(meshes, "q1.3/10O8/7AVC")

In [ ]:
# pygamer.writeDolfin('outputTetMesh.xml', tetmesh)

In [ ]:
toRemove = []
for vertexID in tetmesh.vertexIDs:
    if vertexID.data()[1] > 0.1:
        toRemove.append(vertexID)
for v in toRemove:
    tetmesh.removeVertex(v)
        
surfmesh = tetmesh.extractSurface()
# toRemove = []
# for edgeID in surfmesh.edgeIDs:
#     if len(surfmesh.getCover(edgeID)) < 2:
#         toRemove.append(edgeID)
# for e in toRemove:
#     surfmesh.removeEdge(e)
surfmesh.correctNormals()

v, e, f = surfmesh.to_ndarray()

In [ ]:
# Draw the protein and surrounding box together
ctx = threevis.Context(width=640, height=480)

rgb = np.ones((len(f), 3))

for i, face in enumerate(surfmesh.faceIDs):
    if face.data().marker == 23:
        rgb[i,0] = 1
        rgb[i,1] = 0
        rgb[i,2] = 0
    elif face.data().marker == 50:
        rgb[i,0] = 0
        rgb[i,1] = 0
        rgb[i,2] = 1      
        
colors = threevis.FaceAttribute(rgb)

ctx.draw_faces(v, f, colors=colors)
ctx.draw_edges(v, e)
ctx.display()